## Load Data

* n_initial: 232
* n_completed_survey: 219
* n_completed_and_no_duplicates:
* n_completed_and_no_duplicatesa_and_passed_check:

In [1]:
import pandas as pd

survey_df = pd.read_csv('https://raw.githubusercontent.com/ahmedfaridkhan/item-estimation-survey/main/BA-830-Survey_results.csv', skiprows=[1, 2], header=0)

#Check dataframe
survey_df.head(2)

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,treatment 2,control,Q1,Q2,Q3,Q4,Q5,Q6,Gender,Age
0,2024-02-25 16:51:25,2024-02-25 16:51:49,IP Address,128.197.29.250,25,23,False,2024-02-27 13:17:06,R_1p77grDRCEMPuUh,NaN,...,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-02-25 16:51:37,2024-02-25 16:51:51,IP Address,104.28.39.33,40,14,False,2024-02-27 13:17:06,R_10TP6Y7vnC7qWyZ,NaN,...,NaN,Lars Halden,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Cleaning and Data Preparation

In [2]:
#Check columns
survey_df.columns

Index(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
       'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'LocationLatitude', 'LocationLongitude',
       'DistributionChannel', 'UserLanguage', 'treatment 1', 'treatment 2',
       'control', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Gender', 'Age'],
      dtype='object')

In [3]:
survey_df = survey_df.drop(['StartDate', 'EndDate', 'Status', 'Duration (in seconds)', 'RecordedDate', 'ResponseId', 'RecipientLastName', 'RecipientFirstName',\
       'RecipientEmail', 'ExternalReference', 'LocationLatitude', 'LocationLongitude', 'DistributionChannel', 'UserLanguage'], axis = 1)

In [4]:
survey_df.Finished.value_counts()

True     219
False     13
Name: Finished, dtype: int64

In [5]:
#Keep only people that finished the survey (out of 232)
survey_df = survey_df[survey_df['Finished'] == True]

survey_df = survey_df.reset_index(drop=True)

In [6]:
#Create a new column to check if the person passed the attention question

#Define the function to create 'passed_check' column
def check_values(row):
    if pd.notna(row['treatment 1']):
        return row['treatment 1'] == 9
    elif pd.notna(row['treatment 2']):
        return row['treatment 2'] == 102
    elif pd.notna(row['control']):
        return row['control'] == 'Lars Halden'
    else:
        return False

#Apply function to the survey data
survey_df['passed_check'] = survey_df.apply(check_values, axis=1)

survey_df[['passed_check', 'control','treatment 1', 'treatment 2']].head(5)

,passed_check,control,treatment 1,treatment 2
0,True,NaN,9.0,NaN
1,True,NaN,NaN,102.0
2,True,NaN,NaN,102.0
3,False,Ingrid Johansen,NaN,NaN
4,True,Lars Halden,NaN,NaN


In [7]:
#Assign people to treatment or control
import numpy as np

survey_df['group'] = 'control'  # Set a default value for the 'group' column

for value in range(len(survey_df)):
    if pd.isna(survey_df['treatment 1'].iloc[value]) and pd.isna(survey_df['treatment 2'].iloc[value]):
        survey_df['group'].iloc[value] = 'control'
    elif pd.isna(survey_df['control'].iloc[value]) and pd.isna(survey_df['treatment 2'].iloc[value]):
        survey_df['group'].iloc[value] = 'treatment 1'
    else:
        survey_df['group'].iloc[value] = 'treatment 2'

<ipython-input-7-8607cf2b5fea>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df['group'].iloc[value] = 'treatment 1'
<ipython-input-7-8607cf2b5fea>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df['group'].iloc[value] = 'treatment 2'
<ipython-input-7-8607cf2b5fea>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df['group'].iloc[value] = 'control'


In [8]:
survey_df = survey_df.drop(['Progress', 'Finished', 'treatment 1', 'treatment 2', 'control'], axis = 1)

In [9]:
survey_df

,IPAddress,Q1,Q2,Q3,Q4,Q5,Q6,Gender,Age,passed_check,group
0,77.164.41.88,11.0,160.0,180.0,60.0,280.0,180,Female,22.0,True,treatment 1
1,172.225.170.111,20.0,100.0,70.0,40.0,200.0,150,Male,30.0,True,treatment 2
2,190.111.246.27,10.0,300.0,140.0,50.0,550.0,350,Female,26.0,True,treatment 2
3,67.205.195.178,10.0,500.0,180.0,90.0,350.0,1200,Male,53.0,False,control
4,172.58.134.27,10.0,100.0,120.0,40.0,800.0,90,Female,23.0,True,control
...,...,...,...,...,...,...,...,...,...,...,...
214,177.11.210.150,2.0,100.0,150.0,120.0,400.0,250,Male,42.0,True,control
215,42.2.71.129,13.0,200.0,150.0,120.0,500.0,250,Non-binary / third gender,35.0,True,control
216,186.222.199.176,10.0,50.0,90.0,55.0,220.0,80,Female,16.0,True,treatment 2
217,186.222.199.176,12.0,150.0,208.0,92.0,2400.0,100,Female,67.0,True,treatment 2


In [10]:
survey_df.group.value_counts()

treatment 2    81
control        80
treatment 1    58
Name: group, dtype: int64

In [11]:
#Removing duplicates
survey_df = survey_df.drop_duplicates(subset = ['IPAddress'], keep = 'first')

In [12]:
survey_df.group.value_counts()

treatment 2    73
control        70
treatment 1    49
Name: group, dtype: int64

## Sample Size Calculation

In [13]:
### Calculating Cohen's D 1.
### From experiment: general knowledge question estimations (Seattle temp, GW death age)

mean1 = 0.44
mean2 = -0.43
dims = mean1 - mean2
assumed_std = 1

cohen_d_1 = dims/assumed_std

### Calculating Cohen's D 2.
#From experiment: Same as 1 but with extreme anchors.

mean1 = 0.33
mean2 = -0.33
dims = mean1 - mean2
assumed_std = 1

cohen_d_2 = dims/assumed_std

### Calculating Cohen's D 3.
### From experiment: mean Feb temperatures in Germany

mean1 = 14.89
mean2 = 12.82
dims = mean1 - mean2
std = 2.61

cohen_d_3 = dims/std

### Calculating Cohen's D 4.
### From experiment: Car price experiment

mean1 = 21219
mean2 = 17150
dims = mean1 - mean2
std = 4840

cohen_d_4 = dims/std

#Calculating average cohens d

avg_cohen_d = (cohen_d_1+cohen_d_2+cohen_d_3+cohen_d_4)/4


print(cohen_d_1)
print(cohen_d_2)
print(cohen_d_3)
print(cohen_d_4)
print(avg_cohen_d)

0.87
0.66
0.7931034482758622
0.840702479338843
0.7909514819036764


In [14]:
pip install --upgrade statsmodels


In [15]:
# Statistical Power
from statsmodels.stats.power import TTestPower

true_effect = 1

n = TTestPower() .solve_power(effect_size = avg_cohen_d,\
nobs = None, alpha = .01, power = 0.95, alternative='two-sided')

print (f"Necessary sample size: {np.ceil(n)}")

Necessary sample size: 32.0


## Proportions Z Test

In [16]:
from statsmodels.stats.proportion import proportions_ztest

n_treated = survey_df[(survey_df['group'] == 'treatment 1') | (survey_df['group'] == 'treatment 2')].shape[0]
n = survey_df.shape[0]

_, p_val = proportions_ztest(n_treated, n, value = 2/3)
print(f"The p-value for the proportions test between treatment and control is {p_val:.2} so we fail to reject the null hypothesis of proper randomization")

The p-value for the proportions test between treatment and control is 0.37 so we fail to reject the null hypothesis of proper randomization


## Randomization Checker

In [17]:
pip install statsmodels stargazer

In [18]:
import numpy as np

conditions = [
    survey_df['group'] == 'control',
    survey_df['group'] == 'treatment 1'
]

values = [0, 1]

default_value = 2

survey_df['treatment_arm'] = np.select(conditions, values, default=default_value)
survey_df['Gender_numeric'] = survey_df['Gender'].map({'Male': 1, 'Female': 0, 'Non-binary / third gende': 2})

<ipython-input-18-529e0858e0c7>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df['treatment_arm'] = np.select(conditions, values, default=default_value)
<ipython-input-18-529e0858e0c7>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_df['Gender_numeric'] = survey_df['Gender'].map({'Male': 1, 'Female': 0, 'Non-binary / third gende': 2})


### Control v Treatments

In [19]:
import statsmodels.api as sm
from stargazer.stargazer import Stargazer

model_treatment_arm_gender = sm.OLS.from_formula('Gender_numeric ~ C(treatment_arm)', data=survey_df).fit()
model_treatment_arm_age = sm.OLS.from_formula('Age ~ C(treatment_arm)', data=survey_df).fit()

stargazer = Stargazer([model_treatment_arm_gender, model_treatment_arm_age])
stargazer.custom_columns(['Gender Model', 'Age Model'])

stargazer.show_model_numbers = False
stargazer.show_degrees_of_freedom = False
stargazer.show_r2 = False
stargazer.show_adj_r2 = False
stargazer.show_residual_std_err = False
stargazer.show_f_statistic = False
stargazer.show_nobs = False

stargazer.rename_covariates({'Intercept': 'Control', 'C(treatment_arm)[T.1]':'Treatment 1', 'C(treatment_arm)[T.2]': 'Treatment 2'})
stargazer

### Treatment 1 vs Treatment 2

In [20]:
survey_df_treatmentcheck = survey_df[survey_df['treatment_arm']!=0]

In [21]:
model_treatment_arm_gender = sm.OLS.from_formula('Gender_numeric ~ C(treatment_arm)', data=survey_df_treatmentcheck).fit()
model_treatment_arm_age = sm.OLS.from_formula('Age ~ C(treatment_arm)', data=survey_df_treatmentcheck).fit()

stargazer = Stargazer([model_treatment_arm_gender, model_treatment_arm_age])
stargazer.custom_columns(['Gender Model', 'Age Model'])

stargazer.show_model_numbers = False
stargazer.show_degrees_of_freedom = False
stargazer.show_r2 = False
stargazer.show_adj_r2 = False
stargazer.show_residual_std_err = False
stargazer.show_f_statistic = False
stargazer.show_nobs = False

stargazer.rename_covariates({'Intercept': 'Treatment 1', 'C(treatment_arm)[T.2]': 'Treatment 2'})
stargazer

##Regressions (additive assumptions)

### Regression 1 (All records)

####Installs

In [22]:
!pip install pyfixest
from pyfixest.estimation import feols
from pyfixest.utils import get_data
from pyfixest import etable

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 32.3 MB/s eta 0:00:00


####Data Prep

In [23]:
#Create new DataFrame
df_all = survey_df.copy(deep=True)

In [24]:
#Given that one IPAdress can make multiple estimates we first need to melt the df
melted_df_all = df_all.melt(id_vars=['IPAddress', 'Gender', 'Age', 'group'],
                           value_vars=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6'],
                           var_name='Question',
                           value_name='Score')

melted_df_all.sort_values(by='IPAddress').head(2)

,IPAddress,Gender,Age,group,Question,Score
252,103.49.252.230,Male,23.0,control,Q2,146.0
828,103.49.252.230,Male,23.0,control,Q5,225.0


In [25]:
#Score is an object type, we need to change it to numeric - first to float because one value is to large to convert to C long
melted_df_all['Score'] = melted_df_all['Score'].astype(float)

In [26]:
#Another problem is we have an much too large number for score
print(melted_df_all['Score'].max())

max_value = melted_df_all['Score'].max()

8.392938473802937e+27


In [27]:
#Dropping largest value
indices_to_drop = melted_df_all[melted_df_all['Score'] == max_value].index

#Drop these row(s) from the DataFrame
melted_df_all = melted_df_all.drop(indices_to_drop)

#Converting to int
melted_df_all['Score'] = melted_df_all['Score'].astype(int)

####Model

In [39]:
#Creating the regression

#Regressing the estimage on experiment arm with question(which image they saw)
#as a fixed effect
formula = 'Score ~ 1 + group| Question'

#Adding clustered standard errors to account for the fact that each IPAddress
#made 6 estimates
model = feols(formula, data=melted_df_all, vcov={'CRV1':'IPAddress'})
etable(model)

                                   est1
--------------------  -----------------
depvar                            Score
---------------------------------------
group[T.treatment 1]     9.254 (50.478)
group[T.treatment 2]  -57.508* (27.499)
---------------------------------------
Question                              x
---------------------------------------
R2                                0.179
S.E. type                 by: IPAddress
Observations                       1151
---------------------------------------
Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001
Format of coefficient cell:
Coefficient (Std. Error)


###Regression 2: Only participants who passed the attentio check (`passed_check == True`)

#### Data Prep

In [73]:
#Create new DataFrame
df_passed_check = survey_df.copy(deep=True)

In [74]:
#Keep only the records of people that passed our attention check
df_passed_check = df_passed_check[df_passed_check['passed_check']==True]

In [75]:
#Given that one IPAdress can make multiple estimates we first need to melt the df
melted_df_passed = df_passed_check.melt(id_vars=['IPAddress', 'Gender', 'Age', 'group'],
                           value_vars=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6'],
                           var_name='Question',
                           value_name='Score')

melted_df_passed.sort_values(by='IPAddress').head(2)

,IPAddress,Gender,Age,group,Question,Score
229,103.49.252.230,Male,23.0,control,Q2,146.0
925,103.49.252.230,Male,23.0,control,Q6,176


In [76]:
#Score is an object type, we need to change it to numeric - first to float because one value is to large to convert to C long
melted_df_passed['Score'] = melted_df_passed['Score'].astype(float)

In [77]:
#Another problem is we have an much too large number for score
print(melted_df_passed['Score'].max())

max_value = melted_df_passed['Score'].max()

8.392938473802937e+27


In [78]:
#Dropping largest value
indices_to_drop = melted_df_passed[melted_df_passed['Score'] == max_value].index

#Drop these row(s) from the DataFrame
melted_df_passed = melted_df_passed.drop(indices_to_drop)

#Converting to int
melted_df_passed['Score'] = melted_df_passed['Score'].astype(int)

#### Model

In [79]:
#Creating the regression

#Regressing the estimage on experiment arm with question(which image they saw)
#as a fixed effect
formula = 'Score ~ group| Question'

#Adding clustered standard errors to account for the fact that each IPAddress
#made 6 estimates
model = feols(formula, data=melted_df_passed, vcov={'CRV1':'IPAddress'})
etable(model)

                                  est1
--------------------  ----------------
depvar                           Score
--------------------------------------
group[T.treatment 1]   36.728 (55.585)
group[T.treatment 2]  -48.424 (28.351)
--------------------------------------
Question                             x
--------------------------------------
R2                               0.170
S.E. type                by: IPAddress
Observations                      1043
--------------------------------------
Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001
Format of coefficient cell:
Coefficient (Std. Error)


###Regression 3: Treatment 1 vs Treatment 2

#### Data Prep

In [83]:
melted_df_passed_only_treatment = melted_df_passed\
 [(melted_df_passed['group']=='treatment 1') |(melted_df_passed['group']=='treatment 2')]

In [84]:
melted_df_passed_only_treatment

,IPAddress,Gender,Age,group,Question,Score
0,77.164.41.88,Female,22.0,treatment 1,Q1,11
1,172.225.170.111,Male,30.0,treatment 2,Q1,20
2,190.111.246.27,Female,26.0,treatment 2,Q1,10
5,189.204.251.139,Female,30.0,treatment 2,Q1,12
6,200.173.53.123,Male,26.0,treatment 1,Q1,11
...,...,...,...,...,...,...
1029,191.13.222.67,Female,24.0,treatment 1,Q6,40
1031,176.79.82.16,Male,38.0,treatment 1,Q6,150
1037,187.21.174.215,Male,50.0,treatment 1,Q6,118
1039,66.38.95.243,Female,22.0,treatment 1,Q6,70


#### Model

In [85]:
#Creating the regression

#Regressing the estimage on experiment arm with question(which image they saw)
#as a fixed effect
formula = 'Score ~ group| Question'

#Adding clustered standard errors to account for the fact that each IPAddress
#made 6 estimates
model = feols(formula, data=melted_df_passed_only_treatment, vcov={'CRV1':'IPAddress'})
etable(model)

                                  est1
--------------------  ----------------
depvar                           Score
--------------------------------------
group[T.treatment 2]  -85.152 (51.528)
--------------------------------------
Question                             x
--------------------------------------
R2                               0.176
S.E. type                by: IPAddress
Observations                       654
--------------------------------------
Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001
Format of coefficient cell:
Coefficient (Std. Error)


## Regressions (multiplicative assumptions)

###Regression 4: All participants (log)

####Data Prep

In [61]:
melted_df_all['Score'] = melted_df_all['Score'] + 1

In [62]:
melted_df_all['logScore'] = np.log(melted_df_all['Score'])

#### Model

In [65]:
#Creating the regression

#Regressing the estimage on experiment arm with question(which image they saw)
#as a fixed effect
formula = 'logScore ~ group| Question'

#Adding clustered standard errors to account for the fact that each IPAddress
#made 6 estimates
model = feols(formula, data=melted_df_all, vcov={'CRV1':'IPAddress'})
etable(model)

                                est1
--------------------  --------------
depvar                      logScore
------------------------------------
group[T.treatment 1]  -0.031 (0.100)
group[T.treatment 2]  -0.181 (0.094)
------------------------------------
Question                           x
------------------------------------
R2                             0.668
S.E. type              by: IPAddress
Observations                    1151
------------------------------------
Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001
Format of coefficient cell:
Coefficient (Std. Error)


### Regression 5: Checked participants (log)

####Data Prep

In [66]:
melted_df_passed['Score'] = melted_df_passed['Score'] + 1

In [67]:
melted_df_passed['logScore'] = np.log(melted_df_all['Score'])

#### Model

In [69]:
#Creating the regression

#Regressing the estimage on experiment arm with question(which image they saw)
#as a fixed effect
formula = 'logScore ~ group| Question'

#Adding clustered standard errors to account for the fact that each IPAddress
#made 6 estimates
model = feols(formula, data=melted_df_passed, vcov={'CRV1':'IPAddress'})
etable(model)

                                est1
--------------------  --------------
depvar                      logScore
------------------------------------
group[T.treatment 1]   0.017 (0.065)
group[T.treatment 2]  -0.085 (0.059)
------------------------------------
Question                           x
------------------------------------
R2                             0.566
S.E. type              by: IPAddress
Observations                    1042
------------------------------------
Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001
Format of coefficient cell:
Coefficient (Std. Error)


###Regression 6: Treatment 1 vs Treatment 2 (log)

####Data Prep

In [88]:
melted_df_passed_only_treatment['Score'] = melted_df_passed_only_treatment['Score'] + 1

<ipython-input-88-ccd638a4908d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  melted_df_passed_only_treatment['Score'] = melted_df_passed_only_treatment['Score'] + 1


In [89]:
melted_df_passed_only_treatment['logScore'] = np.log(melted_df_passed_only_treatment['Score'])

<ipython-input-89-5683bfcc8fa6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  melted_df_passed_only_treatment['logScore'] = np.log(melted_df_passed_only_treatment['Score'])


#### Model

In [90]:
#Creating the regression

#Regressing the estimage on experiment arm with question(which image they saw)
#as a fixed effect
formula = 'logScore ~ group| Question'

#Adding clustered standard errors to account for the fact that each IPAddress
#made 6 estimates
model = feols(formula, data=melted_df_passed_only_treatment, vcov={'CRV1':'IPAddress'})
etable(model)

                                est1
--------------------  --------------
depvar                      logScore
------------------------------------
group[T.treatment 2]  -0.197 (0.102)
------------------------------------
Question                           x
------------------------------------
R2                             0.678
S.E. type              by: IPAddress
Observations                     654
------------------------------------
Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001
Format of coefficient cell:
Coefficient (Std. Error)
